<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/Copy_of_CLASSIFICATION1_LLMembed_su1ahn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch transformers tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00


# Data처리

In [42]:
from datasets import load_dataset, ClassLabel, DatasetDict
from sklearn.model_selection import train_test_split

agnews_dataset = load_dataset("fancyzhx/ag_news")
existing_labels = agnews_dataset["train"].features["label"].names
print(f"AG News labels: {existing_labels}")

# 새로운 레이블 생성 - 'StackOverflow' 추가
new_labels = existing_labels + ["StackOverflow"]
new_class_label = ClassLabel(names=new_labels)

# AG News 데이터셋의 'label' 칼럼을 새로운 ClassLabel로 캐스팅
agnews_dataset = agnews_dataset.cast_column("label", new_class_label)

# AG News 'train' 데이터를 'train'과 'validation'으로 분할
# 'train' 데이터를 90%는 'train', 10%는 'validation'으로 분할
agnews_train_valid = agnews_dataset['train'].train_test_split(test_size=0.1, seed=42)
agnews_train_dataset = agnews_train_valid['train']
agnews_validation_dataset = agnews_train_valid['test']
# 분할된 데이터를 포함하도록 agnews_dataset 업데이트
agnews_dataset = DatasetDict({
    'train': agnews_train_dataset,
    'validation': agnews_validation_dataset,
    'test': agnews_dataset['test']
})

# 결과 출력
print(agnews_dataset)

AG News labels: ['World', 'Sports', 'Business', 'Sci/Tech']
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 108000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [44]:
from datasets import load_dataset, ClassLabel, concatenate_datasets, DatasetDict, Features, Value

stackoverflow_dataset = load_dataset(
    'csv',
    data_files={
        'train': '/content/drive/MyDrive/ML/AI-algorithm/project/rev_tag_training_samples.csv',
        'validation': '/content/drive/MyDrive/ML/AI-algorithm/project/rev_tag_validation_samples.csv',
        'test': '/content/drive/MyDrive/ML/AI-algorithm/project/rev_tag_test_samples.csv'
    }
)

# StackOverflow 데이터셋 처리 함수 정의
def process_stackoverflow(example):
    # 'Title'과 'Body'를 결합하여 'text' 생성
    title = example.get('Title', '')
    body = example.get('Body', '')
    text = f"{title} {body}".strip()
    # 'label'을 'StackOverflow'에 해당하는 정수로 설정
    label = new_class_label.str2int('StackOverflow')
    return {'text': text, 'label': label}

# 각 스플릿별로 StackOverflow 데이터셋에 'text'와 'label' 적용 및 불필요한 컬럼 제거
for split in stackoverflow_dataset.keys():
    stackoverflow_dataset[split] = stackoverflow_dataset[split].map(
        process_stackoverflow,
        remove_columns=stackoverflow_dataset[split].column_names
    )


# StackOverflow 데이터셋의 피처를 AG News와 동일하게 설정
for split in stackoverflow_dataset.keys():
    stackoverflow_dataset[split] = stackoverflow_dataset[split].cast(
        Features({
            'text': Value('string'),
            'label': new_class_label
        })
    )

print(stackoverflow_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 6500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})


In [45]:
# 4. 각 스플릿별로 데이터셋 결합
combined_splits = {}
for split in agnews_dataset.keys():
    if split in stackoverflow_dataset:
        combined_splits[split] = concatenate_datasets(
            [agnews_dataset[split], stackoverflow_dataset[split]]
        )
    else:
        combined_splits[split] = agnews_dataset[split]

# 결합된 데이터셋 생성
combined_dataset = DatasetDict(combined_splits)

# 결과 확인
print(combined_dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 128000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10600
    })
})


In [46]:
# 5. 레이블별 데이터 건수 출력
from collections import Counter

for split in combined_dataset.keys():
    labels = combined_dataset[split]['label']
    label_counts = Counter(labels)
    print(f"\n{split} 스플릿의 레이블 분포:")
    for label_int, count in label_counts.items():
        label_name = new_class_label.int2str(label_int)
        print(f"{label_name}: {count}")


train 스플릿의 레이블 분포:
World: 26991
Sci/Tech: 26943
Sports: 26966
Business: 27100
StackOverflow: 20000

validation 스플릿의 레이블 분포:
World: 3009
Sports: 3034
Sci/Tech: 3057
Business: 2900
StackOverflow: 6500

test 스플릿의 레이블 분포:
Business: 1900
Sci/Tech: 1900
Sports: 1900
World: 1900
StackOverflow: 3000


In [47]:
#### Dataset 확인
# 'Sci/Tech' 레이블의 하나의 데이터 출력
sci_tech_label_id = new_class_label.str2int('Sci/Tech')

# 각 스플릿에서 'Sci/Tech' 레이블을 가진 예제 찾기
for split in combined_dataset.keys():
    sci_tech_examples = combined_dataset[split].filter(lambda example: example['label'] == sci_tech_label_id)
    if len(sci_tech_examples) > 0:
        print(f"'Sci/Tech' 레이블의 예제 (스플릿: {split}):")
        print(sci_tech_examples[0])
        break

# 'StackOverflow' 레이블의 하나의 데이터 출력
stackoverflow_label_id = new_class_label.str2int('StackOverflow')

# 각 스플릿에서 'StackOverflow' 레이블을 가진 예제 찾기
for split in combined_dataset.keys():
    stackoverflow_examples = combined_dataset[split].filter(lambda example: example['label'] == stackoverflow_label_id)
    if len(stackoverflow_examples) > 0:
        print(f"\n'StackOverflow' 레이블의 예제 (스플릿: {split}):")
        print(stackoverflow_examples[0])
        break


Filter:   0%|          | 0/128000 [00:00<?, ? examples/s]

'Sci/Tech' 레이블의 예제 (스플릿: train):
{'text': 'Bob Evans, mainframe pioneer, dies at 77 Evans led a team that developed a new class of mainframe computers that allowed different applications to be run simultaneously.', 'label': 3}


Filter:   0%|          | 0/128000 [00:00<?, ? examples/s]


'StackOverflow' 레이블의 예제 (스플릿: train):
{'text': 'Why threads are needed in my given assignment in java? <p><strong>I\'m not asking to do my assignment. Read carefully</strong></p>\n\n<blockquote>\n  <p>Write a program to simulate a bus traveling between 5 different stations and\n  repeats the cycle, the bus can take up to a maximum of 50 persons, at each\n  station random number of persons get off the bus and random number of\n  persons get on the bus, consider these cases.</p>\n  \n  <ul>\n  <li>If bus does not have enough space for all persons, persons will have to\n  stay in station for next cycle</li>\n  <li>Persons cannot mount on bus until persons on bus dismount first.</li>\n  <li>You can simulate bus trip with a fixed delay between each stop to\n  simulate travel time.</li>\n  <li>Persons can not mount/dismount the bus until bus arrives to the\n  designated station.</li>\n  </ul>\n  \n  <p>Use semaphores to control access to the bus and other utilities to control access\n  to b

In [49]:
# combined_dataset의 각 스플릿을 변수에 저장
combined_train_dataset = combined_dataset['train']
combined_validation_dataset = combined_dataset['validation']
combined_test_dataset = combined_dataset['test']

# 새로운 DatasetDict 생성
new_agnews_dataset = DatasetDict({
    "train": combined_train_dataset,
    "validation": combined_validation_dataset,
    "test": combined_test_dataset
})

# 결과 확인
print(new_agnews_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 128000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10600
    })
})


In [ ]:
# from transformers import AutoTokenizer

# new_agnews_dataset= DatasetDict({
#     "train": combined_train_dataset,
#     "test": combined_test_dataset
# })

# # DatasetDict 출력
# print(new_agnews_dataset)

# tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_datasets = new_agnews_dataset.map(tokenize_function, batched=True)



# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1500))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1500))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9500
    })
})


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

# **1. Llama3.2-3B Embedding**

In [50]:
import os
import torch
import json
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
from tqdm import trange
from datasets import load_dataset
import pandas as pd

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [52]:
# -*- coding: utf-8 -*-

def rep_extract(task, mode, device, sents, labels, max_len, step):
    model_id = "unsloth/Llama-3.2-3B-Instruct"

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # padding 토큰 설정 (eos_token을 pad_token으로 설정)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = "right"

    config_kwargs = {
        "trust_remote_code": True,
        "cache_dir": None,
        "revision": 'main',
        "use_auth_token": None,
        "output_hidden_states": True
    }
    model_config = AutoConfig.from_pretrained(model_id, **config_kwargs)
    model = LlamaForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        device_map=device,
        torch_dtype=torch.float16
    )
    model.eval()

    sents_reps = []
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)

            reps_batch_5L = []
            for layer in range(-1, -6, -1):
                reps_batch_5L.append(torch.mean(batch_outputs.hidden_states[layer], axis=1))
            reps_batch_5L = torch.stack(reps_batch_5L, axis=1)

        sents_reps.append(reps_batch_5L.cpu())
    sents_reps = torch.cat(sents_reps)

    labels = torch.tensor(labels)

    path = f'/content/drive/MyDrive/ML/AI-algorithm/project/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    cuda_no = 0 # GPU 번호
    task = 'llama3-2' # Task 이름
    device = f'cuda:{cuda_no}'
    max_len = 512
    step = 32

    # 'train' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['train']['text']
    labels = new_agnews_dataset['train']['label']
    rep_extract(task, 'train', device, sents, labels, max_len, step)

    # 'validation' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['validation']['text']
    labels = new_agnews_dataset['validation']['label']
    rep_extract(task, 'validation', device, sents, labels, max_len, step)

    # 'test' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['test']['text']
    labels = new_agnews_dataset['test']['label']
    rep_extract(task, 'test', device, sents, labels, max_len, step)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


model.safetensors:  55%|#####5    | 3.54G/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

100%|██████████| 4000/4000 [44:32<00:00,  1.50it/s]
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
100%|██████████| 579/579 [06:26<00:00,  1.50it/s]
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
100%|██████████| 332/332 [03:41<00:00,  1.50it/s]


# **2. BERT Embedding**

In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
import json
from transformers import BertTokenizer, BertModel
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels):
    model_path = 'google-bert/bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 512
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            reps_batch = batch_outputs.pooler_output
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/ML/AI-algorithm/project/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':

    cuda_no = 0 # GPU 번호
    task = 'bert' # Task 이름
    device = f'cuda:{cuda_no}'

    # 'train' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['train']['text']
    labels = new_agnews_dataset['train']['label']
    rep_extract(task, 'train', device, sents, labels)

    # 'validation' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['validation']['text']
    labels = new_agnews_dataset['validation']['label']
    rep_extract(task, 'validation', device, sents, labels)

    # 'test' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['test']['text']
    labels = new_agnews_dataset['test']['label']
    rep_extract(task, 'test', device, sents, labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 250/250 [42:54<00:00, 10.30s/it]


torch.Size([128000, 1024])
torch.Size([128000])


 76%|███████▌  | 28/37 [04:48<01:41, 11.25s/it]

# **3. Roberta Embedding**

In [54]:
# -*- coding: utf-8 -*-
import os
import torch
import json
from transformers import RobertaTokenizer, RobertaModel
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels):
    model_path = 'FacebookAI/roberta-large'
    tokenizer = RobertaTokenizer.from_pretrained(model_path)
    model = RobertaModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 512
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            reps_batch = batch_outputs.last_hidden_state[:, 0, :]
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/ML/AI-algorithm/project/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    cuda_no = 0 # GPU 번호
    task = 'roberta' # Task 이름
    device = f'cuda:{cuda_no}'

    # 'train' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['train']['text']
    labels = new_agnews_dataset['train']['label']
    rep_extract(task, 'train', device, sents, labels)

    # 'validation' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['validation']['text']
    labels = new_agnews_dataset['validation']['label']
    rep_extract(task, 'validation', device, sents, labels)

    # 'test' 데이터셋에 대한 임베딩 추출
    sents = new_agnews_dataset['test']['text']
    labels = new_agnews_dataset['test']['label']
    rep_extract(task, 'test', device, sents, labels)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

torch.Size([128000, 1024])
torch.Size([128000])


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 37/37 [06:50<00:00, 11.10s/it]


torch.Size([18500, 1024])
torch.Size([18500])


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 21/21 [03:55<00:00, 11.19s/it]

torch.Size([10600, 1024])
torch.Size([10600])


In [100]:
import torch

# 파일 경로 설정
l_path = '/content/drive/MyDrive/ML/AI-algorithm/project/llama3-2/dataset_tensor/'
b_path = '/content/drive/MyDrive/ML/AI-algorithm/project/bert/dataset_tensor/'
r_path = '/content/drive/MyDrive/ML/AI-algorithm/project/roberta/dataset_tensor/'
mode = 'train'  # train, test, validation 등의 모드에 따라 변경 가능

# 각 pt 파일의 임베딩 로드 및 크기 확인
l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
b_sents_reps = torch.load(b_path + f'{mode}_sents.pt')
r_sents_reps = torch.load(r_path + f'{mode}_sents.pt')

print(f"L Embedding Shape: {l_sents_reps.shape}")
print(f"B Embedding Shape: {b_sents_reps.shape}")
print(f"R Embedding Shape: {r_sents_reps.shape}")


<ipython-input-100-f89f6534d2a9>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
<ipython-input-100-f89f6534d2a9>:11

L Embedding Shape: torch.Size([128000, 5, 3072])
B Embedding Shape: torch.Size([128000, 1024])
R Embedding Shape: torch.Size([128000, 1024])


In [101]:
from torch.utils.data import Dataset, DataLoader

class UnifiedDataset(Dataset):
    def __init__(self, l_path, b_path, r_path, mode, max_length=512):
        self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt').mean(dim=1)  # [128000, 5, 3072] -> [128000, 3072]
        self.b_sents_reps = torch.load(b_path + f'{mode}_sents.pt')  # [128000, 1024]
        self.r_sents_reps = torch.load(r_path + f'{mode}_sents.pt')  # [128000, 1024]
        self.labels = torch.load(l_path + f'{mode}_labels.pt')
        self.sample_num = self.labels.shape[0]
        self.max_length = max_length

    def __getitem__(self, index):
        l_embedding = self.l_sents_reps[index]
        b_embedding = self.b_sents_reps[index]
        r_embedding = self.r_sents_reps[index]
        label = self.labels[index]

        return l_embedding, b_embedding, r_embedding, label

    def __len__(self):
        return self.sample_num


In [106]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel

class UnifiedModel(nn.Module):
    def __init__(self, class_num, model_checkpoint, embedding_dim=3072, hidden_size=1024, output_size=256, dropout_rate=0.5, sigma=0.1):
        super(UnifiedModel, self).__init__()
        self.sigma = sigma
        # compress_layers의 입력 크기를 embedding_dim(3072)로 설정하고, 출력 크기를 hidden_size(1024)로 설정
        self.compress_layers = nn.ModuleList([nn.Sequential(
            nn.Linear(embedding_dim, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)) for _ in range(3)])

        # B와 R 임베딩을 3072 차원으로 맞추는 Linear 레이어 추가
        self.b_linear = nn.Linear(1024, 3072)
        self.r_linear = nn.Linear(1024, 3072)

        # EFT 기반 feature extractor
        self.language_model = AutoModel.from_pretrained(model_checkpoint)
        self.classifier_head = ExtremeFineTuningClassificationHead(
            input_size=hidden_size,
            hidden_size=output_size,
            output_size=class_num,
            activation_function=nn.PReLU(init=0.5),
            dropout=nn.Dropout(dropout_rate)
        )

        # fc1의 입력 크기를 hidden_size * 3으로 설정하여 compress_layers 출력과 맞춥니다.
        self.fc1 = nn.Linear(hidden_size * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc3 = nn.Linear(output_size, class_num)
        self.softmax = nn.Softmax(dim=1)

        # I-ELM 모드 플래그 설정
        self.elm_mode = False

    def forward(self, input_l, input_b, input_r):
        #batch_size = input_l.shape[0]

        # 입력 텐서를 float32로 변환
        input_l, input_b, input_r = input_l.float(), input_b.float(), input_r.float()

        # B와 R 임베딩을 3072 차원으로 변환
        input_b = self.b_linear(input_b)
        input_r = self.r_linear(input_r)

        # LLMEmbed 다중 임베딩 처리
        embedded_inputs = [layer(embed) for layer, embed in zip(self.compress_layers, [input_l, input_b, input_r])]
        embedded_inputs = torch.cat(embedded_inputs, dim=1)

        # 통합 입력으로 결합
        x = self.fc1(embedded_inputs)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return self.softmax(x)

    def set_elm_mode(self):
        """I-ELM 모드를 활성화합니다."""
        self.elm_mode = True

    def unset_elm_mode(self):
        """I-ELM 모드를 비활성화합니다."""
        self.elm_mode = False

    def step_helper(self, input_l, input_b, input_r, labels):
        """I-ELM용 입력 데이터 전처리"""
        combined_features = self.forward(input_l, input_b, input_r)
        labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=self.fc3.out_features).float()
        return combined_features, labels_one_hot

    def _iterative_elm(self, input_features, label_one_hot):
        """I-ELM을 사용하여 가중치 업데이트 수행"""
        if not hasattr(self, 'H'):
            self.H = input_features.T @ input_features
            self.T = input_features.T @ label_one_hot
            self.output_weights = torch.linalg.pinv(self.H) @ self.T
        else:
            self.H += input_features.T @ input_features
            self.T += input_features.T @ label_one_hot
            self.output_weights = torch.linalg.pinv(self.H) @ self.T

In [89]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoModel

# class UnifiedModel(nn.Module):
#     def __init__(self, class_num, model_checkpoint, embedding_dim=3072, hidden_size=1024, output_size=256, dropout_rate=0.5, sigma=0.1):
#         super(UnifiedModel, self).__init__()
#         self.sigma = sigma
#         self.compress_layers = nn.ModuleList([nn.Sequential(
#             nn.Linear(embedding_dim, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)) for _ in range(3)])

#         # EFT 기반 feature extractor
#         self.language_model = AutoModel.from_pretrained(model_checkpoint)
#         self.classifier_head = ExtremeFineTuningClassificationHead(
#             input_size=hidden_size,
#             hidden_size=output_size,
#             output_size=class_num,
#             activation_function=nn.PReLU(init=0.5),
#             dropout=nn.Dropout(dropout_rate)
#         )

#         self.fc1 = nn.Linear(hidden_size * 3, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, output_size)
#         self.fc3 = nn.Linear(output_size, class_num)
#         self.softmax = nn.Softmax(dim=1)

#         # I-ELM 모드 플래그 설정
#         self.elm_mode = False

#     def forward(self, input_l, input_b, input_r):
#         batch_size = input_l.shape[0]

#         # LLMEmbed 다중 임베딩 처리
#         input_l = input_l.float()
#         input_b = input_b.float()
#         input_r = input_r.float()

#         embedded_inputs = [layer(embed) for layer, embed in zip(self.compress_layers, [input_l, input_b, input_r])]
#         embedded_inputs = torch.cat(embedded_inputs, dim=1)

#         # 통합 입력으로 결합
#         x = self.fc1(embedded_inputs)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.fc3(x)
#         return self.softmax(x)

#     def set_elm_mode(self):
#         """I-ELM 모드를 활성화합니다."""
#         self.elm_mode = True

#     def unset_elm_mode(self):
#         """I-ELM 모드를 비활성화합니다."""
#         self.elm_mode = False

#     def step_helper(self, input_l, input_b, input_r, labels):
#         """I-ELM용 입력 데이터 전처리"""
#         combined_features = self.forward(input_l, input_b, input_r)
#         labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=self.fc3.out_features).float()
#         return combined_features, labels_one_hot

#     def _iterative_elm(self, input_features, label_one_hot):
#         """I-ELM을 사용하여 가중치 업데이트 수행"""
#         if not hasattr(self, 'H'):
#             self.H = input_features.T @ input_features
#             self.T = input_features.T @ label_one_hot
#             self.output_weights = torch.linalg.pinv(self.H) @ self.T
#         else:
#             self.H += input_features.T @ input_features
#             self.T += input_features.T @ label_one_hot
#             self.output_weights = torch.linalg.pinv(self.H) @ self.T

In [107]:
class ExtremeFineTuningClassificationHead(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_function=None, dropout=None):
        super(ExtremeFineTuningClassificationHead, self).__init__()
        self.input_weights = nn.Linear(input_size, hidden_size)
        self.output_weights = nn.Linear(hidden_size, output_size)
        self.activation_function = activation_function if activation_function else nn.Tanh()
        self.dropout = dropout

    def forward(self, features):
        x = features
        if self.dropout:
            x = self.dropout(x)
        x = self.input_weights(x)
        x = self.activation_function(x)
        if self.dropout:
            x = self.dropout(x)
        x = self.output_weights(x)
        return x

In [108]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
#from unified_fine_tuning import UnifiedModel, UnifiedDataset

def set_hyperparameters():
    """하이퍼파라미터 설정 함수"""
    # EFT 하이퍼파라미터
    eft_params = {
        'hidden_size': 1024,
        'output_size': 256,
        'dropout_rate': 0.1,
        'lr': 1e-4  # EFT 학습률
    }

    # LLMEmbed 하이퍼파라미터
    llmembed_params = {
        'SIGMA': 0.1,
        'dropout_rate': 0.5,
        'compress_layers': 5,  # LLM 임베딩 레이어 수
        'lr': 5e-5  # LLMEmbed 학습률
    }

    # 최적화 설정 조합
    combined_params = {
        'total_epochs': 10,
        'elm_epochs': 3,  # I-ELM 학습을 수행할 초기 에폭 수
        'batch_size': 32,
        'num_classes': 5  # 예시로 5개 클래스 설정, 데이터셋에 따라 조정
    }
    return eft_params, llmembed_params, combined_params

def train_model(model, dataloader, device, loss_fn, optimizer, eft_optimizer, epochs=10, elm_epochs=3):
    """모델 학습 함수"""
    for epoch in range(epochs):
        total_loss, total_acc = 0, 0

        if epoch < elm_epochs:
            print(f"Epoch {epoch+1}/{epochs} - Using I-ELM training")
            model.set_elm_mode()  # 모델을 I-ELM 모드로 설정
            with torch.no_grad():  # I-ELM은 경사 하강을 사용하지 않음
                for batch in dataloader:
                    input_l, input_b, input_r, labels = batch
                    input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

                    # I-ELM 가중치 업데이트
                    features, labels_one_hot = model.step_helper(input_l, input_b, input_r, labels)
                    model._iterative_elm(features, labels_one_hot)

            model.unset_elm_mode()  # I-ELM 모드 해제

        else:
            print(f"Epoch {epoch+1}/{epochs} - Using Backpropagation training")
            model.train()
            for batch in dataloader:
                input_l, input_b, input_r, labels = batch
                input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

                # Forward 및 손실 계산
                optimizer.zero_grad()
                eft_optimizer.zero_grad()  # EFT 파라미터의 옵티마이저
                outputs = model(input_l, input_b, input_r)
                loss = loss_fn(outputs, labels)

                # Backward
                loss.backward()
                optimizer.step()
                eft_optimizer.step()  # EFT 파라미터 업데이트

                # Loss 및 정확도 기록
                total_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                total_acc += (preds == labels).sum().item() / len(labels)

            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}, Accuracy: {total_acc / len(dataloader):.4f}")

def evaluate_model(model, dataloader, device):
    model.eval()
    total_acc, total_loss = 0, 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for input_l, input_b, input_r, labels in dataloader:
            input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)
            outputs = model(input_l, input_b, input_r)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            total_acc += (preds == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / (len(dataloader.dataset))
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {avg_acc:.4f}")



if __name__ == '__main__':
    # Hyperparameters 설정
    eft_params, llmembed_params, combined_params = set_hyperparameters()
    #model_checkpoint = 'roberta-base'  # EFT에서 사용하는 사전 학습 모델
    model_checkpoint = 'FacebookAI/roberta-large'
    l_path = '/content/drive/MyDrive/ML/AI-algorithm/project/llama3-2/dataset_tensor/'  # LLM 임베딩 경로
    b_path = '/content/drive/MyDrive/ML/AI-algorithm/project/bert/dataset_tensor/'
    r_path = '/content/drive/MyDrive/ML/AI-algorithm/project/roberta/dataset_tensor/'
    batch_size = combined_params['batch_size']
    total_epochs = combined_params['total_epochs']
    elm_epochs = combined_params['elm_epochs']
    num_classes = combined_params['num_classes']

    # Device 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 데이터셋 및 데이터로더 초기화
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    train_dataset = UnifiedDataset(l_path=l_path, b_path=b_path, r_path=r_path, mode='train')
    test_dataset = UnifiedDataset(l_path=l_path, b_path=b_path, r_path=r_path, mode='test')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # 모델 초기화
    model = UnifiedModel(
        class_num=num_classes,
        model_checkpoint=model_checkpoint,
        hidden_size=eft_params['hidden_size'],
        output_size=eft_params['output_size'],
        dropout_rate=eft_params['dropout_rate'],
        sigma=llmembed_params['SIGMA']
    )
    model.to(device)

    # 손실 함수 및 옵티마이저
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=llmembed_params['lr'])  # LLMEmbed 학습률 사용
    eft_optimizer = optim.AdamW(model.language_model.parameters(), lr=eft_params['lr'])  # EFT 학습률 사용

    # 모델 학습
    print("Training the model...")
    train_model(model, train_loader, device, loss_fn, optimizer, eft_optimizer, epochs=total_epochs, elm_epochs=elm_epochs)

    # 모델 평가
    print("Evaluating the model...")
    evaluate_model(model, test_loader, device)


<ipython-input-101-23972395ebc1>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt').mean(dim=1)  # [128000, 5, 3072

Training the model...
Epoch 1/10 - Using I-ELM training
Epoch 2/10 - Using I-ELM training
Epoch 3/10 - Using I-ELM training
Epoch 4/10 - Using Backpropagation training
Epoch 4/10, Loss: 0.9895, Accuracy: 0.9179
Epoch 5/10 - Using Backpropagation training
Epoch 5/10, Loss: 0.9697, Accuracy: 0.9348
Epoch 6/10 - Using Backpropagation training
Epoch 6/10, Loss: 0.9673, Accuracy: 0.9371
Epoch 7/10 - Using Backpropagation training
Epoch 7/10, Loss: 0.9662, Accuracy: 0.9382
Epoch 8/10 - Using Backpropagation training
Epoch 8/10, Loss: 0.9650, Accuracy: 0.9393
Epoch 9/10 - Using Backpropagation training
Epoch 9/10, Loss: 0.9638, Accuracy: 0.9405
Epoch 10/10 - Using Backpropagation training
Epoch 10/10, Loss: 0.9626, Accuracy: 0.9417
Evaluating the model...
Test Loss: 0.9597, Test Accuracy: 0.9446


# **4. Dataset 정의**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

    def __init__(self, mode, l_path, b_path, r_path):
        self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
        self.b_sents_reps = torch.load(b_path + f'{mode}_sents.pt')
        self.r_sents_reps = torch.load(r_path + f'{mode}_sents.pt')

        self.labels = torch.load(l_path + f'{mode}_labels.pt')

        self.sample_num = self.labels.shape[0]

    def __getitem__(self, index):
        return self.l_sents_reps[index], self.b_sents_reps[index], self.r_sents_reps[index], self.labels[index]

    def __len__(self):
        return self.sample_num

# **5. Model operation (multi)**

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import numpy as np

# epoch - Train
def Train_multi(dataloader, device, model, loss_fn, optimizer):
    loss_list, acc_list, micro_f1_list, macro_f1_list = [], [], [], []

    for batch_i, batch_loader in enumerate(tqdm(dataloader)):
        batch_l, batch_b, batch_r, batch_y = batch_loader
        batch_l, batch_b, batch_r, batch_y = batch_l.to(device), batch_b.to(device), batch_r.to(device), batch_y.to(device)

        model.train()
        pred = model(batch_l.float(), batch_b.float(), batch_r.float())
        loss = loss_fn(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_y = torch.max(pred, 1).indices
        # print(pred_y)
        acc = accuracy_score(batch_y.cpu(), pred_y.cpu())
        micro_f1 = f1_score(batch_y.cpu(), pred_y.cpu(), average='micro')
        macro_f1 = f1_score(batch_y.cpu(), pred_y.cpu(), average='macro')
        loss = loss.cpu()

        loss_list.append(loss.item())
        acc_list.append(acc)
        micro_f1_list.append(micro_f1)
        macro_f1_list.append(macro_f1)
    print(f'loss: {np.mean(loss_list):.4f}')
    print(f'acc: {np.mean(acc_list):.4f}')
    print(f'micro_f1: {np.mean(micro_f1_list):.4f}')
    print(f'macro_f1: {np.mean(macro_f1_list):.4f}')

def Test_multi(dataloader, device, model, loss_fn):
    avg_loss = 0
    total_pred, total_y = [], []

    for batch_i, batch_loader in enumerate(tqdm(dataloader)):
        batch_l, batch_b, batch_r, batch_y = batch_loader
        batch_l, batch_b, batch_r, batch_y = batch_l.to(device), batch_b.to(device), batch_r.to(device), batch_y.to(device)

        model.eval()
        with torch.no_grad():
            pred = model(batch_l.float(), batch_b.float(), batch_r.float())
            loss = loss_fn(pred, batch_y)
            loss = loss.to('cpu')
            avg_loss += loss.item()

        pred_y = torch.max(pred, 1).indices
        total_pred.append(pred_y.cpu())
        total_y.append(batch_y.cpu())

    avg_loss = avg_loss / (batch_i+1)

    total_y = torch.cat(total_y)
    total_pred = torch.cat(total_pred)
    acc = accuracy_score(total_y, total_pred)
    micro_f1 = f1_score(total_y.cpu(), total_pred.cpu(), average='micro')
    macro_f1 = f1_score(total_y.cpu(), total_pred.cpu(), average='macro')

    print(f'avg loss: {avg_loss:.4f}')
    print(f'acc: {acc:.4f}')
    print(f'micro_f1: {micro_f1:.4f}')
    print(f'macro_f1: {macro_f1:.4f}')

# **6. Downstream Model Class**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DownstreamModel(nn.Module):
    def __init__(self, class_num, SIGMA):
        super(DownstreamModel, self).__init__()
        self.SIGMA = SIGMA
        self.compress_layers = nn.ModuleList()
        for _ in range(5):
            layers = []
            layers.append(nn.Linear(4096, 1024))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.5))
            self.compress_layers.append(nn.Sequential(*layers))

        self.fc1 = nn.Linear(4145, 1024)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, class_num)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_l, input_b, input_r):
        batch_size = input_l.shape[0]
        split_tensors = torch.split(input_l, 1, dim=1)
        input = []
        for i, split_tensor in enumerate(split_tensors):
            split_tensor = split_tensor.reshape(batch_size,-1)
            input.append(self.compress_layers[i](split_tensor))
        input.append(input_b)
        input.append(input_r)
        input = torch.stack(input, dim=1)
        # X * X^T
        input_T = input.transpose(1, 2)
        input_P = torch.matmul(input, input_T)
        input_P = input_P.reshape(batch_size, -1)
        # PN func
        input_P = 2*F.sigmoid(self.SIGMA * input_P) - 1

        a = torch.mean(input_l, dim=1)
        input = torch.cat([input_P, a], dim=1)
        # print(input.shape)

        output = self.fc1(input)
        output = self.relu1(output)
        output = self.dropout1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
        output = self.fc3(output)
        output = self.softmax(output)

        return output

# if __name__ == '__main__':
#     model = DownstreamModel(2)
#     print(model)

# **7. Fine-tuning**

In [ ]:
# from DownstreamModel import DownstreamModel
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
# from model_op import Train, Test
# from model_op_multi import Train_multi, Test_multi
import argparse
import os
import torch
# from MyDataset import MyDataset
import json

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cuda_no = 0  # 예를 들어, '0'을 사용
    task = 'agnews'  # 사용 가능한 옵션: 'sst2', 'mr', 'agnews', 'r8', 'r52' 중 하나
    epoches = 100  # 원하는 에폭 수
    SIGMA = 0.1  # SIGMA 값을 설정
    batch_size = 1024  # 배치 크기 설정
    lr = 1e-4  # 학습률 설정

    class_num = {'sst2':2, 'mr':2, 'agnews':5, 'r8':8, 'r52':52}
    class_num = class_num[task]

    l_dataset_path = f'/content/drive/MyDrive/LLMEmbed/llama2/{task}/dataset_tensor/'
    b_dataset_path = f'/content/drive/MyDrive/LLMEmbed/bert/{task}/dataset_tensor/'
    r_dataset_path = f'/content/drive/MyDrive/LLMEmbed/roberta/{task}/dataset_tensor/'
    mode = 'train'
    train_data = MyDataset(mode, l_dataset_path, b_dataset_path, r_dataset_path)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    mode = 'test'
    test_data = MyDataset(mode, l_dataset_path, b_dataset_path, r_dataset_path)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    model = DownstreamModel(class_num, SIGMA).to(device)

    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr)

    if class_num == 2:
        print('training ...')
        for epoch in range(epoches):
            model = model.to(device)
            print(f'--------------------------- epoch {epoch} ---------------------------')
            Train(train_loader, device, model, loss_fn, optimizer)
        print('save model_wights')
        # 모델 학습 후, 모델의 가중치 저장하기
        model_save_path = "/content/drive/MyDrive/LLMEmbed/model_weights.pth"
        torch.save(model.state_dict(), model_save_path)
        print('evaluate ...')
        Test(test_loader, device, model, loss_fn)

    # multi-class
    elif class_num > 2:
        print('training ...')
        for epoch in range(epoches):
            model = model.to(device)
            print(f'--------------------------- epoch {epoch} ---------------------------')
            Train_multi(train_loader, device, model, loss_fn, optimizer)
        print()
        # 모델 학습 후, 모델의 가중치 저장하기
        model_save_path = "/content/drive/MyDrive/LLMEmbed/model_weights_agnews.pth"
        torch.save(model.state_dict(), model_save_path)
        print('evaluate ...')
        Test_multi(test_loader, device, model, loss_fn)

<ipython-input-1-9044650157bc>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
<ipython-input-1-9044650157bc>:8:

training ...
--------------------------- epoch 0 ---------------------------


100%|██████████| 147/147 [00:05<00:00, 27.81it/s]


loss: 1.0586
acc: 0.8725
micro_f1: 0.8725
macro_f1: 0.8697
--------------------------- epoch 1 ---------------------------


100%|██████████| 147/147 [00:05<00:00, 27.90it/s]


...

loss: 0.9255
acc: 0.9795
micro_f1: 0.9795
macro_f1: 0.9794
--------------------------- epoch 99 ---------------------------


100%|██████████| 147/147 [00:05<00:00, 27.35it/s]


loss: 0.9249
acc: 0.9801
micro_f1: 0.9801
macro_f1: 0.9800

evaluate ...


100%|██████████| 10/10 [00:01<00:00,  7.49it/s]

avg loss: 0.9410
acc: 0.9608
micro_f1: 0.9608
macro_f1: 0.9608


# **8. 추론 메서드 정의**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from transformers import AutoConfig

# Load the tokenizers and models for Llama2, BERT, and Roberta
llama2_tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf", trust_remote_code=True)
llama2_tokenizer.pad_token = llama2_tokenizer.eos_token  # 패딩 토큰 설정
llama2_config = AutoConfig.from_pretrained("daryl149/llama-2-7b-chat-hf", output_hidden_states=True)
llama2_model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf", config=llama2_config)

bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
bert_model = BertModel.from_pretrained('google-bert/bert-large-uncased')

roberta_tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-large')
roberta_model = RobertaModel.from_pretrained('FacebookAI/roberta-large')

# Make sure all models are in evaluation mode and moved to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llama2_model.eval().to(device)
bert_model.eval().to(device)
roberta_model.eval().to(device)

# Initialize the downstream model
class_num = 5  # For example, if you have 8 classes
SIGMA = 0.1  # SIGMA value for your downstream model
downstream_model = DownstreamModel(class_num, SIGMA).to(device)

model_load_path = "/content/drive/MyDrive/LLMEmbed/model_weights_agnews.pth"

# 가중치 로드
downstream_model.load_state_dict(torch.load(model_save_path, map_location=device))
downstream_model.eval()

def get_llama2_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Average the last 5 layers
        embedding = torch.stack([torch.mean(outputs.hidden_states[i], dim=1) for i in range(-1, -6, -1)], dim=1)
    return embedding

def get_bert_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use pooler_output for BERT embeddings
        embedding = outputs.pooler_output
    return embedding

def get_roberta_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use the first token ([CLS] token) representation
        embedding = outputs.last_hidden_state[:, 0, :]
    return embedding

def infer(text, downstream_model, device):
    # 각 모델로부터 임베딩을 추출
    llama2_emb = get_llama2_embedding(text, llama2_tokenizer, llama2_model, device)
    bert_emb = get_bert_embedding(text, bert_tokenizer, bert_model, device)
    roberta_emb = get_roberta_embedding(text, roberta_tokenizer, roberta_model, device)

    # Forward pass through the downstream model
    with torch.no_grad():
        prediction = downstream_model(llama2_emb, bert_emb, roberta_emb)

    return prediction


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-23-dde813e42baf>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com

# **9. 추론 테스트**

In [ ]:
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech",
    4: "StackOverFlow"
}

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "US trade deficit swells in June The US trade deficit has exploded 19 to a record \$55.8bn as oil costs drove imports higher, according to a latest figures.",
    "Indians Beat Twins 7-1, Nearing AL Lead (AP) AP - The Cleveland Indians pulled within one game of the AL Central lead, scoring four runs in the first inning and beating the Minnesota Twins 7-1 Saturday night behind home runs by Travis Hafner and Victor Martinez.",
    "Pilgrims Crowd Field for Mass With Pope LOURDES, France - A frail Pope John Paul II celebrated an open-air Mass on Sunday as several hundred thousand pilgrims, many in wheelchairs, crowded onto a field near a French shrine to the Virgin Mary that is associated with miraculous cures of the sick. The Mass was a highlight of the Pope's two-day visit to Lourdes, a town in the Pyrenees where Roman Catholic tradition says St...",
    "T. Rex Had Teen Growth Spurt, Scientists Say (Reuters) Reuters - Tyrannosaurus Rex grew incredibly fast\during a teenaged growth spurt that saw the dinosaur expand its\bulk by six times, but the fearsome beasts \"lived fast and died\young,\" researchers said on Wednesday.",
    "working of compareTo() method of Comparable interface            I have one Employee class and the requirement is to sort the objects using comparable interface. The output with this code is :The difference of this id and other id is..** 6  other id**1The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**1The difference of this id and other id is..** 11  other id**3The difference of this id and other id is..** 11  other id**6",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = infer(text, downstream_model, device)
    predicted_label = torch.argmax(prediction, dim=1).item()
    predicted_score = torch.max(prediction).item()
    label_name = label_map[predicted_label]
    print(f"Text {i+1} Predicted label: {label_name}, Score: {predicted_score:.4f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text 1 Predicted label: Business, Score: 1.0000
Text 2 Predicted label: Sports, Score: 0.9954
Text 3 Predicted label: World, Score: 1.0000
Text 4 Predicted label: Sci/Tech, Score: 1.0000
Text 5 Predicted label: StackOverFlow, Score: 1.0000
